# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f31c49d6730>
├── 'a' --> tensor([[-0.7600, -1.1657, -0.3402],
│                   [ 0.9653, -1.6865,  0.8421]])
└── 'x' --> <FastTreeValue 0x7f3125000ee0>
    └── 'c' --> tensor([[ 1.3378, -1.1356, -0.9695, -0.8551],
                        [-1.4599, -0.8781, -0.8647, -0.3310],
                        [ 1.5181,  0.7818, -0.9724,  0.4350]])

In [4]:
t.a

tensor([[-0.7600, -1.1657, -0.3402],
        [ 0.9653, -1.6865,  0.8421]])

In [5]:
%timeit t.a

87.2 ns ± 0.419 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f31c49d6730>
├── 'a' --> tensor([[ 0.4515,  0.3554, -0.4237],
│                   [ 2.3273, -0.2693,  0.9679]])
└── 'x' --> <FastTreeValue 0x7f3125000ee0>
    └── 'c' --> tensor([[ 1.3378, -1.1356, -0.9695, -0.8551],
                        [-1.4599, -0.8781, -0.8647, -0.3310],
                        [ 1.5181,  0.7818, -0.9724,  0.4350]])

In [7]:
%timeit t.a = new_value

90.8 ns ± 1.31 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7600, -1.1657, -0.3402],
               [ 0.9653, -1.6865,  0.8421]]),
    x: Batch(
           c: tensor([[ 1.3378, -1.1356, -0.9695, -0.8551],
                      [-1.4599, -0.8781, -0.8647, -0.3310],
                      [ 1.5181,  0.7818, -0.9724,  0.4350]]),
       ),
)

In [10]:
b.a

tensor([[-0.7600, -1.1657, -0.3402],
        [ 0.9653, -1.6865,  0.8421]])

In [11]:
%timeit b.a

76.2 ns ± 0.451 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6909, -0.8973, -0.0474],
               [ 0.2616,  0.4364,  0.8003]]),
    x: Batch(
           c: tensor([[ 1.3378, -1.1356, -0.9695, -0.8551],
                      [-1.4599, -0.8781, -0.8647, -0.3310],
                      [ 1.5181,  0.7818, -0.9724,  0.4350]]),
       ),
)

In [13]:
%timeit b.a = new_value

633 ns ± 8.81 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 15.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.3 µs ± 95.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

163 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

167 µs ± 423 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f30f9c461f0>
├── 'a' --> tensor([[[-0.7600, -1.1657, -0.3402],
│                    [ 0.9653, -1.6865,  0.8421]],
│           
│                   [[-0.7600, -1.1657, -0.3402],
│                    [ 0.9653, -1.6865,  0.8421]],
│           
│                   [[-0.7600, -1.1657, -0.3402],
│                    [ 0.9653, -1.6865,  0.8421]],
│           
│                   [[-0.7600, -1.1657, -0.3402],
│                    [ 0.9653, -1.6865,  0.8421]],
│           
│                   [[-0.7600, -1.1657, -0.3402],
│                    [ 0.9653, -1.6865,  0.8421]],
│           
│                   [[-0.7600, -1.1657, -0.3402],
│                    [ 0.9653, -1.6865,  0.8421]],
│           
│                   [[-0.7600, -1.1657, -0.3402],
│                    [ 0.9653, -1.6865,  0.8421]],
│           
│                   [[-0.7600, -1.1657, -0.3402],
│                    [ 0.9653, -1.6865,  0.8421]]])
└── 'x' --> <FastTreeValue 0x7f30f9c46340>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.8 µs ± 116 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f30f9c46490>
├── 'a' --> tensor([[-0.7600, -1.1657, -0.3402],
│                   [ 0.9653, -1.6865,  0.8421],
│                   [-0.7600, -1.1657, -0.3402],
│                   [ 0.9653, -1.6865,  0.8421],
│                   [-0.7600, -1.1657, -0.3402],
│                   [ 0.9653, -1.6865,  0.8421],
│                   [-0.7600, -1.1657, -0.3402],
│                   [ 0.9653, -1.6865,  0.8421],
│                   [-0.7600, -1.1657, -0.3402],
│                   [ 0.9653, -1.6865,  0.8421],
│                   [-0.7600, -1.1657, -0.3402],
│                   [ 0.9653, -1.6865,  0.8421],
│                   [-0.7600, -1.1657, -0.3402],
│                   [ 0.9653, -1.6865,  0.8421],
│                   [-0.7600, -1.1657, -0.3402],
│                   [ 0.9653, -1.6865,  0.8421]])
└── 'x' --> <FastTreeValue 0x7f30f9c464f0>
    └── 'c' --> tensor([[ 1.3378, -1.1356, -0.9695, -0.8551],
                        [-1.4599, -0.8781, -0.8647, -0.3310],
                 

In [23]:
%timeit t_cat(trees)

35.8 µs ± 120 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

78.3 µs ± 678 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.3378, -1.1356, -0.9695, -0.8551],
                       [-1.4599, -0.8781, -0.8647, -0.3310],
                       [ 1.5181,  0.7818, -0.9724,  0.4350]],
              
                      [[ 1.3378, -1.1356, -0.9695, -0.8551],
                       [-1.4599, -0.8781, -0.8647, -0.3310],
                       [ 1.5181,  0.7818, -0.9724,  0.4350]],
              
                      [[ 1.3378, -1.1356, -0.9695, -0.8551],
                       [-1.4599, -0.8781, -0.8647, -0.3310],
                       [ 1.5181,  0.7818, -0.9724,  0.4350]],
              
                      [[ 1.3378, -1.1356, -0.9695, -0.8551],
                       [-1.4599, -0.8781, -0.8647, -0.3310],
                       [ 1.5181,  0.7818, -0.9724,  0.4350]],
              
                      [[ 1.3378, -1.1356, -0.9695, -0.8551],
                       [-1.4599, -0.8781, -0.8647, -0.3310],
                       [ 1.5181,  0.7818, -0.9724,  0.4350]],

In [26]:
%timeit Batch.stack(batches)

95.8 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.3378, -1.1356, -0.9695, -0.8551],
                      [-1.4599, -0.8781, -0.8647, -0.3310],
                      [ 1.5181,  0.7818, -0.9724,  0.4350],
                      [ 1.3378, -1.1356, -0.9695, -0.8551],
                      [-1.4599, -0.8781, -0.8647, -0.3310],
                      [ 1.5181,  0.7818, -0.9724,  0.4350],
                      [ 1.3378, -1.1356, -0.9695, -0.8551],
                      [-1.4599, -0.8781, -0.8647, -0.3310],
                      [ 1.5181,  0.7818, -0.9724,  0.4350],
                      [ 1.3378, -1.1356, -0.9695, -0.8551],
                      [-1.4599, -0.8781, -0.8647, -0.3310],
                      [ 1.5181,  0.7818, -0.9724,  0.4350],
                      [ 1.3378, -1.1356, -0.9695, -0.8551],
                      [-1.4599, -0.8781, -0.8647, -0.3310],
                      [ 1.5181,  0.7818, -0.9724,  0.4350],
                      [ 1.3378, -1.1356, -0.9695, -0.8551],
                   

In [28]:
%timeit Batch.cat(batches)

171 µs ± 1.83 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

393 µs ± 8.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
